In [1]:
### data wrangling - combining files into one single file (one line of information per each user)

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()
from sklearn.preprocessing import OrdinalEncoder

In [3]:
# Reading the Credit Bureau file from csv
bureau = pd.read_csv('bureau.csv', index_col=None, engine='python')

In [4]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [5]:
# Reading the Bureau Balance file from csv
bureau_balance = pd.read_csv('bureau_balance.csv', index_col=None, engine='python')

In [6]:
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [7]:
enc = OrdinalEncoder()
bureau_balance[["STATUS"]] = enc.fit_transform(bureau_balance[["STATUS"]])

In [8]:
bureau_balance = bureau_balance.drop("MONTHS_BALANCE", axis=1)

In [9]:
bureau_balance.head()

,SK_ID_BUREAU,STATUS
0,5715448,6.0
1,5715448,6.0
2,5715448,6.0
3,5715448,6.0
4,5715448,6.0


In [10]:
counts = bureau_balance['SK_ID_BUREAU'].value_counts().to_dict()


In [11]:
bureau_balance_grouped = bureau_balance.groupby(['SK_ID_BUREAU'], as_index=False).mean()

In [12]:
bureau_balance_grouped.head()

,SK_ID_BUREAU,STATUS
0,5001709,6.113402
1,5001710,6.000000
2,5001711,1.750000
3,5001712,2.842105
4,5001713,7.000000


In [13]:
bureau_balance_grouped['Month_Balance_Count'] = bureau_balance_grouped['SK_ID_BUREAU'].map(counts)
bureau_balance_grouped.head()

,SK_ID_BUREAU,STATUS,Month_Balance_Count
0,5001709,6.113402,97
1,5001710,6.000000,83
2,5001711,1.750000,4
3,5001712,2.842105,19
4,5001713,7.000000,22


In [14]:
# Back to the Credit Bureau file - transforming categorical variables into Dummies

enc = OrdinalEncoder()
bureau[["CREDIT_ACTIVE","CREDIT_CURRENCY", "CREDIT_TYPE"]] = enc.fit_transform(bureau[["CREDIT_ACTIVE","CREDIT_CURRENCY", "CREDIT_TYPE"]])


bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,2.0,0.0,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,3.0,-131,NaN
1,215354,5714463,0.0,0.0,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,4.0,-20,NaN
2,215354,5714464,0.0,0.0,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,3.0,-16,NaN
3,215354,5714465,0.0,0.0,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,4.0,-16,NaN
4,215354,5714466,0.0,0.0,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,3.0,-21,NaN


In [15]:
bureau.shape

(1716428, 17)

In [16]:
# Merging the Credit Bureau file with the Balance file
bureau = bureau.merge(bureau_balance_grouped, on=('SK_ID_BUREAU'), how='left')

In [17]:
bureau.shape

(1716428, 19)

In [18]:
bureau_grouped = bureau.groupby(['SK_ID_CURR'], as_index=False).sum()

In [19]:
bureau_grouped.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,STATUS,Month_Balance_Count
0,100001,41276431,8.0,0.0,-5145,0,577.0,-3302.0,0.000,0,1453365.000,596686.5,0.000,0.0,21.0,-652,24817.5,29.099630,172.0
1,100002,49226177,12.0,0.0,-6992,0,-2094.0,-4185.0,8405.145,0,865055.565,245781.0,31988.565,0.0,28.0,-3999,0.0,19.534091,110.0
2,100003,23543514,6.0,0.0,-5603,0,-2178.0,-3292.0,0.000,0,1017400.500,0.0,810000.000,0.0,14.0,-3264,0.0,0.000000,0.0
3,100004,13658267,4.0,0.0,-1734,0,-977.0,-1065.0,0.000,0,189037.800,0.0,0.000,0.0,6.0,-1064,0.0,0.000000,0.0
4,100005,20205603,2.0,0.0,-572,0,1318.0,-123.0,0.000,0,657126.000,568408.5,0.000,0.0,10.0,-163,4261.5,5.179487,21.0


In [20]:
bureau_grouped.shape

(305811, 19)

In [21]:
#Validating unique ID count against the size of final grouped DataFrame
bureau_grouped['SK_ID_CURR'].nunique()

305811

In [22]:
bureau_grouped = bureau_grouped.drop(['SK_ID_BUREAU'], axis=1)

In [23]:
# Reading the Credit Card Balance file from csv
cc_balance = pd.read_csv('credit_card_balance.csv', index_col=None, engine='python')

In [24]:
cc_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [25]:
# Transforming categorical variable into Dummies

cc_balance[["NAME_CONTRACT_STATUS"]] = enc.fit_transform(cc_balance[["NAME_CONTRACT_STATUS"]])

cc_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,0.0,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,0.0,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,0.0,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,0.0,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,0.0,0,0


In [26]:
cc_balance.shape

(3840312, 23)

In [27]:
cc_balance['SK_ID_PREV'].nunique()

104307

In [28]:
# Creating unique combinations of the previous loans IDs with the current loan IDs
cc_balance_prev_curr = cc_balance[['SK_ID_PREV','SK_ID_CURR']]
cc_balance_prev_curr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840312 entries, 0 to 3840311
Data columns (total 2 columns):
 #   Column      Dtype
---  ------      -----
 0   SK_ID_PREV  int64
 1   SK_ID_CURR  int64
dtypes: int64(2)
memory usage: 58.6 MB


In [29]:
cc_balance_prev_curr = cc_balance_prev_curr.groupby(['SK_ID_PREV'], as_index=False).mean()
cc_balance_prev_curr.tail()

,SK_ID_PREV,SK_ID_CURR
104302,2843476,197090
104303,2843477,168439
104304,2843478,424526
104305,2843493,337804
104306,2843496,425374


In [30]:
# Dropping Current ID variable - later will re-merge
cc_balance = cc_balance.drop("SK_ID_CURR", axis=1)

In [31]:
# Summing past Installement payment information and grouping by previous ID.
cc_balance = cc_balance.groupby(['SK_ID_PREV'], as_index=False).sum()
cc_balance.head()

,SK_ID_PREV,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1000018,-20,374731.425,405000,27000.0,147394.980,0.0,120394.980,12970.44,27708.75,...,368012.925,368012.925,6.0,44,0.0,38.0,10.0,0.0,0,0
1,1000030,-36,447928.515,652500,4500.0,138059.505,0.0,133559.505,16625.79,43320.42,...,447483.015,447483.015,1.0,41,0.0,40.0,15.0,0.0,0,0
2,1000031,-136,838311.030,2394000,157500.0,463353.840,0.0,305853.840,42804.81,354519.09,...,833599.530,833599.530,4.0,21,0.0,17.0,59.0,0.0,0,0
3,1000035,-20,0.000,1125000,0.0,0.000,0.0,0.000,0.00,0.00,...,0.000,0.000,0.0,0,0.0,0.0,0.0,0.0,0,0
4,1000077,-77,0.000,1035000,0.0,0.000,0.0,0.000,0.00,0.00,...,0.000,0.000,0.0,0,0.0,0.0,0.0,0.0,0,0


In [32]:
# Merging back the unique SK_ID_CURR variable to the installments_payments DataFrame
cc_balance = cc_balance.merge(cc_balance_prev_curr, on=('SK_ID_PREV'), how='left')
cc_balance.head()

,SK_ID_PREV,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,...,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,SK_ID_CURR
0,1000018,-20,374731.425,405000,27000.0,147394.980,0.0,120394.980,12970.44,27708.75,...,368012.925,6.0,44,0.0,38.0,10.0,0.0,0,0,394447
1,1000030,-36,447928.515,652500,4500.0,138059.505,0.0,133559.505,16625.79,43320.42,...,447483.015,1.0,41,0.0,40.0,15.0,0.0,0,0,361282
2,1000031,-136,838311.030,2394000,157500.0,463353.840,0.0,305853.840,42804.81,354519.09,...,833599.530,4.0,21,0.0,17.0,59.0,0.0,0,0,131335
3,1000035,-20,0.000,1125000,0.0,0.000,0.0,0.000,0.00,0.00,...,0.000,0.0,0,0.0,0.0,0.0,0.0,0,0,436351
4,1000077,-77,0.000,1035000,0.0,0.000,0.0,0.000,0.00,0.00,...,0.000,0.0,0,0.0,0.0,0.0,0.0,0,0,181153


In [33]:
cc_balance['SK_ID_PREV'].nunique()

104307

In [34]:
# Reading the Previous Applications file from csv
prev_app = pd.read_csv('previous_application.csv', index_col=None, engine='python')

In [35]:
prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
prev_app.dropna(subset=["WEEKDAY_APPR_PROCESS_START","NAME_CONTRACT_TYPE","NAME_SELLER_INDUSTRY","NAME_YIELD_GROUP","PRODUCT_COMBINATION","FLAG_LAST_APPL_PER_CONTRACT"], inplace = True)


In [37]:
prev_app["FLAG_LAST_APPL_PER_CONTRACT"].unique()


array(['Y', 'N'], dtype=object)

In [38]:
# Previous Applications - transforming categorical variables into Dummies
prev_app[["WEEKDAY_APPR_PROCESS_START","NAME_CONTRACT_TYPE","NAME_SELLER_INDUSTRY","NAME_YIELD_GROUP","PRODUCT_COMBINATION","FLAG_LAST_APPL_PER_CONTRACT"]] = enc.fit_transform(prev_app[["WEEKDAY_APPR_PROCESS_START","NAME_CONTRACT_TYPE","NAME_SELLER_INDUSTRY","NAME_YIELD_GROUP","PRODUCT_COMBINATION","FLAG_LAST_APPL_PER_CONTRACT"]])
prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,1.0,1730.430,17145.0,17145.0,0.0,17145.0,2.0,15,...,2.0,12.0,4.0,13.0,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,0.0,25188.615,607500.0,679671.0,NaN,607500.0,4.0,11,...,10.0,36.0,2.0,7.0,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,0.0,15060.735,112500.0,136444.5,NaN,112500.0,5.0,11,...,10.0,12.0,1.0,6.0,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,0.0,47041.335,450000.0,470790.0,NaN,450000.0,1.0,7,...,10.0,12.0,4.0,8.0,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,0.0,31924.395,337500.0,404055.0,NaN,337500.0,4.0,9,...,10.0,24.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
prev_app.shape

(1669868, 37)

In [40]:
prev_app['SK_ID_PREV'].nunique()

1669868

prev_app has the same number of unique SK_ID_PREV identifiers as there are rows in the DataFrame

In [41]:
# Reading the Previous Applications file from csv
pos_cash_balance = pd.read_csv('POS_CASH_BALANCE.csv', index_col=None, engine='python')

In [42]:
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [43]:
# Previous Applications - transforming categorical variables into Dummies

pos_cash_balance[["NAME_CONTRACT_STATUS"]] = enc.fit_transform(pos_cash_balance[["NAME_CONTRACT_STATUS"]])


pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,0.0,0,0
1,1715348,367990,-33,36.0,35.0,0.0,0,0
2,1784872,397406,-32,12.0,9.0,0.0,0,0
3,1903291,269225,-35,48.0,42.0,0.0,0,0
4,2341044,334279,-35,36.0,35.0,0.0,0,0


In [44]:
pos_cash_balance.shape

(10001358, 8)

In [45]:
pos_cash_balance['SK_ID_PREV'].nunique()

936325

pos cash balance has the same number of unique SK_ID_PREV identifiers as there are rows in the DataFrame

In [46]:
# Reading the Installments Payments file from csv
installments_payments = pd.read_csv('installments_payments.csv', index_col=None, engine='python')

In [47]:
installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [48]:
print(installments_payments.shape)

(13605401, 8)


In [49]:
# Creating unique combinations of the previous loans IDs with the current loan IDs
installments_payments_prev_curr = installments_payments[['SK_ID_PREV','SK_ID_CURR']]
installments_payments_prev_curr = installments_payments_prev_curr.groupby(['SK_ID_PREV'], as_index=False).mean()
installments_payments_prev_curr.tail()

,SK_ID_PREV,SK_ID_CURR
997747,2843495,260963
997748,2843496,425374
997749,2843497,451578
997750,2843498,393881
997751,2843499,314148


In [50]:
# Dropping Current ID variable - later will re-merge
installments_payments = installments_payments.drop("SK_ID_CURR", axis=1)


In [51]:
# New Variable for a flag of a late payment relative to when it was supposed to have been paid
installments_payments['late'] = installments_payments['DAYS_ENTRY_PAYMENT'] > installments_payments['DAYS_INSTALMENT']
installments_payments['late'] = installments_payments['late'].astype(int)

In [52]:
# Summing past Installement payment information and grouping by previous ID.
installments_payments = installments_payments.groupby(['SK_ID_PREV'], as_index=False).sum()
installments_payments.head()

,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,late
0,1000001,3.0,3,-506.0,-538.0,68443.425,68443.425,0
1,1000002,5.0,10,-6220.0,-6299.0,37235.565,37235.565,0
2,1000003,3.0,6,-192.0,-238.0,14854.050,14854.050,0
3,1000004,8.0,28,-5404.0,-5591.0,33523.155,33523.155,0
4,1000005,11.0,64,-16978.0,-17071.0,161735.310,147021.705,2


In [53]:
installments_payments.shape

(997752, 8)

In [54]:
installments_payments['SK_ID_PREV'].nunique()

997752

In [55]:
# Merging back the unique SK_ID_CURR variable to the installments_payments DataFrame
installments_payments = installments_payments.merge(installments_payments_prev_curr, on=('SK_ID_PREV'), how='left')
installments_payments.head()

,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,late,SK_ID_CURR
0,1000001,3.0,3,-506.0,-538.0,68443.425,68443.425,0,158271
1,1000002,5.0,10,-6220.0,-6299.0,37235.565,37235.565,0,101962
2,1000003,3.0,6,-192.0,-238.0,14854.050,14854.050,0,252457
3,1000004,8.0,28,-5404.0,-5591.0,33523.155,33523.155,0,260094
4,1000005,11.0,64,-16978.0,-17071.0,161735.310,147021.705,2,176456


Installments payments now has a unique SK_ID_PREV for every row

In [56]:
#Dropping Curr ID from merged files to keep a single column
pos_cash_balance = pos_cash_balance.drop(['SK_ID_CURR'], axis=1)
installments_payments = installments_payments.drop(['SK_ID_CURR'], axis=1)
cc_balance = cc_balance.drop(['SK_ID_CURR'], axis=1)

In [57]:
# Merging prev_app with installments_payments
prev_app_and_installments = prev_app.merge(installments_payments, on=('SK_ID_PREV'), how='left')
prev_app_and_installments.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,late
0,2030495,271877,1.0,1730.430,17145.0,17145.0,0.0,17145.0,2.0,15,...,-42.0,-37.0,0.0,2.0,1.0,-42.0,-42.0,17284.275,17284.275,0.0
1,2802425,108129,0.0,25188.615,607500.0,679671.0,NaN,607500.0,4.0,11,...,365243.0,365243.0,1.0,5.0,15.0,-370.0,-416.0,125943.075,125943.075,0.0
2,2523466,122040,0.0,15060.735,112500.0,136444.5,NaN,112500.0,5.0,11,...,365243.0,365243.0,1.0,9.0,45.0,-1359.0,-1433.0,135546.615,135546.615,1.0
3,2819243,176158,0.0,47041.335,450000.0,470790.0,NaN,450000.0,1.0,7,...,-182.0,-177.0,1.0,12.0,66.0,-3652.0,-3730.0,563133.375,563133.375,0.0
4,1784265,202054,0.0,31924.395,337500.0,404055.0,NaN,337500.0,4.0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# Merging prev_app+installments_payments with pos_cash_balance
prev_app_and_installments_pos_cash = prev_app_and_installments.merge(pos_cash_balance, on=('SK_ID_PREV'), how='left')
prev_app_and_installments_pos_cash.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,late,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS_y,SK_DPD,SK_DPD_DEF
0,2030495,271877,1.0,1730.430,17145.0,17145.0,0.0,17145.0,2.0,15,...,-42.0,17284.275,17284.275,0.0,-1.0,1.0,0.0,4.0,0.0,0.0
1,2030495,271877,1.0,1730.430,17145.0,17145.0,0.0,17145.0,2.0,15,...,-42.0,17284.275,17284.275,0.0,-2.0,12.0,12.0,0.0,0.0,0.0
2,2802425,108129,0.0,25188.615,607500.0,679671.0,NaN,607500.0,4.0,11,...,-416.0,125943.075,125943.075,0.0,-4.0,36.0,34.0,0.0,0.0,0.0
3,2802425,108129,0.0,25188.615,607500.0,679671.0,NaN,607500.0,4.0,11,...,-416.0,125943.075,125943.075,0.0,-5.0,36.0,35.0,0.0,0.0,0.0
4,2802425,108129,0.0,25188.615,607500.0,679671.0,NaN,607500.0,4.0,11,...,-416.0,125943.075,125943.075,0.0,-3.0,36.0,33.0,0.0,0.0,0.0


In [59]:
# Merging prev_app+installments_payments+pos_cash_balance with cc_balance
all_prev_data = prev_app_and_installments_pos_cash.merge(cc_balance, on=('SK_ID_PREV'), how='left')
all_prev_data.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD_y,SK_DPD_DEF_y
0,2030495,271877,1.0,1730.430,17145.0,17145.0,0.0,17145.0,2.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2030495,271877,1.0,1730.430,17145.0,17145.0,0.0,17145.0,2.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2802425,108129,0.0,25188.615,607500.0,679671.0,NaN,607500.0,4.0,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2802425,108129,0.0,25188.615,607500.0,679671.0,NaN,607500.0,4.0,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2802425,108129,0.0,25188.615,607500.0,679671.0,NaN,607500.0,4.0,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
# Checking that removal of Curr_ID Columns worked and we have a single Curr_ID Column for subsequent merging
all_prev_data.columns = all_prev_data.columns.map(str)
filter_col = all_prev_data.loc[:, all_prev_data.columns.str.startswith('SK_ID_CURR')]
filter_col.head()

,SK_ID_CURR
0,271877
1,271877
2,108129
3,108129
4,108129


In [61]:
filter_col['SK_ID_CURR'].nunique()

338857

In [62]:
all_prev_data['SK_ID_CURR'].nunique()

338857

In [63]:
all_prev_data.shape

(10431762, 71)

In [64]:
# Summing past Installement payment information and grouping by Current ID (Combining info on past transactions)
all_prev_data_by_ID_CURR = all_prev_data.groupby(['SK_ID_CURR'], as_index=False).sum()
all_prev_data_by_ID_CURR = all_prev_data_by_ID_CURR.drop(['SK_ID_PREV'], axis=1)
all_prev_data_by_ID_CURR.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD_y,SK_DPD_DEF_y
0,100001,5.0,19755.000,124177.5,118935.0,12600.0,124177.5,0.0,65,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100002,19.0,175783.725,3402045.0,3402045.0,0.0,3402045.0,38.0,171,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100003,20.0,1384245.000,10725714.0,11892798.0,82620.0,10725714.0,48.0,412,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100004,4.0,21429.000,97128.0,80424.0,19440.0,97128.0,0.0,20,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100005,11.0,52945.200,490792.5,441688.5,49104.0,490792.5,44.0,131,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
all_prev_data_by_ID_CURR.shape

(338857, 60)

In [66]:
# Read Train Data Set
application_train = pd.read_csv('application_train.csv', index_col=None, engine='python')

In [67]:
application_test = pd.read_csv('application_test.csv', index_col=None, engine='python')

In [68]:
# Merging grouped previous transactions and bureau files with the train data set
application_train_merged = application_train.merge(all_prev_data_by_ID_CURR, on=('SK_ID_CURR'), how='left')
application_train_merged = application_train_merged.merge(bureau_grouped, on=('SK_ID_CURR'), how='left')
application_train_merged.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,...,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,STATUS,Month_Balance_Count
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,865055.565,245781.0,31988.565,0.0,28.0,-3999.0,0.0,19.534091,110.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,1017400.500,0.0,810000.000,0.0,14.0,-3264.0,0.0,0.000000,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,189037.800,0.0,0.000,0.0,6.0,-1064.0,0.0,0.000000,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,146250.000,0.0,0.000,0.0,3.0,-783.0,0.0,0.000000,0.0


In [69]:
application_test_merged = application_test.merge(all_prev_data_by_ID_CURR, on=('SK_ID_CURR'), how='left')
application_test_merged = application_test_merged.merge(bureau_grouped, on=('SK_ID_CURR'), how='left')
application_test_merged.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,...,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,STATUS,Month_Balance_Count
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0.0,1453365.00,596686.5,0.00,0.0,21.0,-652.0,24817.500,29.099630,172.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0.0,657126.00,568408.5,0.00,0.0,10.0,-163.0,4261.500,5.179487,21.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0.0,2072280.06,0.0,0.00,0.0,8.0,-3102.0,0.000,16.766930,230.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0.0,1520875.08,186304.5,101390.76,0.0,41.0,-7818.0,21084.075,47.958127,560.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now looking at missing data to decide which columns to drop and which to fill in data for

In [70]:
# renaming a duplicate named variable
cols = []
count = 1
for column in application_train_merged.columns:
    if column == 'AMT_ANNUITY_x':
        cols.append('AMT_ANNUITY_x_'+str(count))
        count+=1
        continue
    cols.append(column)
application_train_merged.columns = cols

In [71]:
# renaming a duplicate named variable
cols = []
count = 1
for column in application_test_merged.columns:
    if column == 'AMT_ANNUITY_x':
        cols.append('AMT_ANNUITY_x_'+str(count))
        count+=1
        continue
    cols.append(column)
application_test_merged.columns = cols

In [72]:
# renaming a duplicate named variable
cols = []
count = 1
for column in application_train_merged.columns:
    if column == 'AMT_ANNUITY_y':
        cols.append('AMT_ANNUITY_y_'+str(count))
        count+=1
        continue
    cols.append(column)
application_train_merged.columns = cols

In [73]:
# renaming a duplicate named variable
cols = []
count = 1
for column in application_test_merged.columns:
    if column == 'AMT_ANNUITY_y':
        cols.append('AMT_ANNUITY_y_'+str(count))
        count+=1
        continue
    cols.append(column)
application_test_merged.columns = cols

In [74]:
# Finding the number of missing variables by column 

def Missing_variables(df):
    nan_values = []
    for i in df:
        count_nan = len(df[i]) - df[i].count()
        nan_values.append(count_nan)
    percentage = [x / len(df)*100 for x in nan_values]
    nan_df = list(df.columns.values)
    data_type=df.dtypes
    percentage_list = pd.DataFrame(
    {'columns': nan_df,
     'number_Nan': nan_values,
     'percentage': percentage,
     'dtype':data_type
    })
    
    Nan_Ascending = percentage_list.loc[percentage_list['percentage'] != 0].sort_values('percentage', ascending=False)
    
    return Nan_Ascending

In [75]:
# Before removing variables with high Nan values, making exception to EXT_SOURCE_1
# This is after seeing its high feature importance to the model
application_train_merged['EXT_SOURCE_1'].fillna((application_train_merged['EXT_SOURCE_1'].mean()), inplace=True)

In [76]:
application_test_merged['EXT_SOURCE_1'].fillna((application_test_merged['EXT_SOURCE_1'].mean()), inplace=True)

In [77]:
Missing_var_train = Missing_variables(application_train_merged)
print('There are', len(Missing_var_train), 'Training columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var_train.head(10)

There are 142 Training columns with missing variables out of 198 columns in the dataframe


,columns,number_Nan,percentage,dtype
COMMONAREA_MEDI,COMMONAREA_MEDI,214865,69.872297,float64
COMMONAREA_MODE,COMMONAREA_MODE,214865,69.872297,float64
COMMONAREA_AVG,COMMONAREA_AVG,214865,69.872297,float64
NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MEDI,213514,69.432963,float64
NONLIVINGAPARTMENTS_MODE,NONLIVINGAPARTMENTS_MODE,213514,69.432963,float64
NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_AVG,213514,69.432963,float64
FONDKAPREMONT_MODE,FONDKAPREMONT_MODE,210295,68.386172,object
LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_MEDI,210199,68.354953,float64
LIVINGAPARTMENTS_MODE,LIVINGAPARTMENTS_MODE,210199,68.354953,float64
LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_AVG,210199,68.354953,float64


In [78]:
# Creating list of column names for columns with over 50% of missing variables
# Removing these columns from the application_train_merged data set

Mising_var_high = Missing_var_train[Missing_var_train['percentage']>35]
high_Nan_column_names = Mising_var_high['columns'].tolist()

application_train_merged = application_train_merged.drop(high_Nan_column_names, axis=1)
application_test_merged = application_test_merged.drop(high_Nan_column_names, axis=1)

In [79]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(20)

There are 94 columns with missing variables out of 150 columns in the dataframe


,columns,number_Nan,percentage,dtype
OCCUPATION_TYPE,OCCUPATION_TYPE,96391,31.345545,object
EXT_SOURCE_3,EXT_SOURCE_3,60965,19.825307,float64
Month_Balance_Count,Month_Balance_Count,44020,14.314935,float64
AMT_CREDIT_SUM,AMT_CREDIT_SUM,44020,14.314935,float64
CREDIT_CURRENCY,CREDIT_CURRENCY,44020,14.314935,float64
DAYS_CREDIT,DAYS_CREDIT,44020,14.314935,float64
CREDIT_DAY_OVERDUE,CREDIT_DAY_OVERDUE,44020,14.314935,float64
DAYS_CREDIT_ENDDATE,DAYS_CREDIT_ENDDATE,44020,14.314935,float64
DAYS_ENDDATE_FACT,DAYS_ENDDATE_FACT,44020,14.314935,float64
AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_MAX_OVERDUE,44020,14.314935,float64


In [80]:
# The majority of missing values are 5.350703 percentage, and those appear on those that did not merge 
# with the applicaiton_train data.
# Since there is much incomplete data - those 5% of the data will be removed for the model data-sets
# Using "late" variable for the removal. 

application_train_merged = application_train_merged[np.isfinite(application_train_merged['late'])]
application_test_merged = application_test_merged[np.isfinite(application_test_merged['late'])]

In [81]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(10)

There are 33 columns with missing variables out of 150 columns in the dataframe


,columns,number_Nan,percentage,dtype
OCCUPATION_TYPE,OCCUPATION_TYPE,91240,31.347812,object
EXT_SOURCE_3,EXT_SOURCE_3,57188,19.648385,float64
CREDIT_ACTIVE,CREDIT_ACTIVE,41550,14.275554,float64
AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_DEBT,41550,14.275554,float64
DAYS_CREDIT_ENDDATE,DAYS_CREDIT_ENDDATE,41550,14.275554,float64
DAYS_ENDDATE_FACT,DAYS_ENDDATE_FACT,41550,14.275554,float64
AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_MAX_OVERDUE,41550,14.275554,float64
CNT_CREDIT_PROLONG,CNT_CREDIT_PROLONG,41550,14.275554,float64
AMT_CREDIT_SUM,AMT_CREDIT_SUM,41550,14.275554,float64
AMT_CREDIT_SUM_OVERDUE,AMT_CREDIT_SUM_OVERDUE,41550,14.275554,float64


In [82]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(10)

There are 33 columns with missing variables out of 150 columns in the dataframe


,columns,number_Nan,percentage,dtype
OCCUPATION_TYPE,OCCUPATION_TYPE,91240,31.347812,object
EXT_SOURCE_3,EXT_SOURCE_3,57188,19.648385,float64
CREDIT_ACTIVE,CREDIT_ACTIVE,41550,14.275554,float64
AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_DEBT,41550,14.275554,float64
DAYS_CREDIT_ENDDATE,DAYS_CREDIT_ENDDATE,41550,14.275554,float64
DAYS_ENDDATE_FACT,DAYS_ENDDATE_FACT,41550,14.275554,float64
AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_MAX_OVERDUE,41550,14.275554,float64
CNT_CREDIT_PROLONG,CNT_CREDIT_PROLONG,41550,14.275554,float64
AMT_CREDIT_SUM,AMT_CREDIT_SUM,41550,14.275554,float64
AMT_CREDIT_SUM_OVERDUE,AMT_CREDIT_SUM_OVERDUE,41550,14.275554,float64


In [83]:
# Filling NaN values in OCCUPATION_TYPE as "Unemployed"
application_train_merged['OCCUPATION_TYPE'].fillna('Unemployed', inplace=True)

# Filling NaN values in EXT_Source_3 and _2 with the mean of each.
application_train_merged['EXT_SOURCE_3'].fillna((application_train_merged['EXT_SOURCE_3'].mean()), inplace=True)
application_train_merged['EXT_SOURCE_2'].fillna((application_train_merged['EXT_SOURCE_2'].mean()), inplace=True)

# Filling NaN values in NAME_TYPE_SUITE as "Unaccompanied"
application_train_merged['NAME_TYPE_SUITE'].fillna('Unaccompanied', inplace=True)

# Filling NaN values in XXX_CNT_SOCIAL_CIRCLE as "0.0". Making assumption that not listed is not observed.
# These are defined as "How many observation of client's social surroundings defaulted on 30 DPD (days past due)""
application_train_merged['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(0.0, inplace=True)
application_train_merged['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(0.0, inplace=True)
application_train_merged['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(0.0, inplace=True)
application_train_merged['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(0.0, inplace=True)

# Filling NaN values in EXT_Source_3 and _2 with the mean of each.
application_train_merged['AMT_ANNUITY_x_1'].fillna((application_train_merged['AMT_ANNUITY_x_1'].mean()), inplace=True)

In [84]:
# Filling NaN values in OCCUPATION_TYPE as "Unemployed"
application_test_merged['OCCUPATION_TYPE'].fillna('Unemployed', inplace=True)

# Filling NaN values in EXT_Source_3 and _2 with the mean of each.
application_test_merged['EXT_SOURCE_3'].fillna((application_test_merged['EXT_SOURCE_3'].mean()), inplace=True)
application_test_merged['EXT_SOURCE_2'].fillna((application_test_merged['EXT_SOURCE_2'].mean()), inplace=True)

# Filling NaN values in NAME_TYPE_SUITE as "Unaccompanied"
application_test_merged['NAME_TYPE_SUITE'].fillna('Unaccompanied', inplace=True)

# Filling NaN values in XXX_CNT_SOCIAL_CIRCLE as "0.0". Making assumption that not listed is not observed.
# These are defined as "How many observation of client's social surroundings defaulted on 30 DPD (days past due)""
application_test_merged['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(0.0, inplace=True)
application_test_merged['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(0.0, inplace=True)
application_test_merged['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(0.0, inplace=True)
application_test_merged['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(0.0, inplace=True)

# Filling NaN values in EXT_Source_3 and _2 with the mean of each.
application_test_merged['AMT_ANNUITY_x_1'].fillna((application_test_merged['AMT_ANNUITY_x_1'].mean()), inplace=True)

In [85]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(24)

There are 24 columns with missing variables out of 150 columns in the dataframe


,columns,number_Nan,percentage,dtype
DAYS_ENDDATE_FACT,DAYS_ENDDATE_FACT,41550,14.275554,float64
AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_MAX_OVERDUE,41550,14.275554,float64
STATUS,STATUS,41550,14.275554,float64
AMT_ANNUITY,AMT_ANNUITY,41550,14.275554,float64
DAYS_CREDIT_UPDATE,DAYS_CREDIT_UPDATE,41550,14.275554,float64
CREDIT_TYPE,CREDIT_TYPE,41550,14.275554,float64
AMT_CREDIT_SUM_OVERDUE,AMT_CREDIT_SUM_OVERDUE,41550,14.275554,float64
AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_LIMIT,41550,14.275554,float64
AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_DEBT,41550,14.275554,float64
AMT_CREDIT_SUM,AMT_CREDIT_SUM,41550,14.275554,float64


In [86]:
def fill_zero(columns):
    for col in columns:
        application_train_merged[col].fillna(0.0, inplace=True)
        application_test_merged[col].fillna(0.0, inplace=True)


In [87]:
Missing_var['columns'].values

array(['DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'STATUS',
       'AMT_ANNUITY', 'DAYS_CREDIT_UPDATE', 'CREDIT_TYPE',
       'AMT_CREDIT_SUM_OVERDUE', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM', 'CNT_CREDIT_PROLONG',
       'Month_Balance_Count', 'DAYS_CREDIT_ENDDATE', 'CREDIT_DAY_OVERDUE',
       'DAYS_CREDIT', 'CREDIT_CURRENCY', 'CREDIT_ACTIVE',
       'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_YEAR',
       'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_MON',
       'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_DAY',
       'AMT_GOODS_PRICE_x'], dtype=object)

In [88]:
fill_zero(Missing_var['columns'].values)

In [89]:
Missing_var = Missing_variables(application_test_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_test_merged.columns), 'columns in the dataframe')
Missing_var.head(10)

There are 0 columns with missing variables out of 149 columns in the dataframe


,columns,number_Nan,percentage,dtype


In [90]:
application_train_merged.shape

(291057, 150)

In [91]:
application_test_merged.shape

(47800, 149)

### Looking at a couple of variables to check whether the data is reasonable


In [93]:
# Age variable
(application_train_merged['DAYS_BIRTH'] / -365).describe()

count    291057.000000
mean         43.975076
std          11.933689
min          20.517808
25%          34.087671
50%          43.189041
75%          53.931507
max          69.043836
Name: DAYS_BIRTH, dtype: float64

In [94]:
(application_train_merged['DAYS_BIRTH'] / -365).describe()

count    291057.000000
mean         43.975076
std          11.933689
min          20.517808
25%          34.087671
50%          43.189041
75%          53.931507
max          69.043836
Name: DAYS_BIRTH, dtype: float64

### The age variable looks to be populated with no unreasonable outliers


In [95]:
(application_train_merged['DAYS_EMPLOYED'] / -365).describe()

count    291057.000000
mean       -175.424256
std         387.581130
min       -1000.665753
25%           0.816438
50%           3.369863
75%           7.594521
max          49.073973
Name: DAYS_EMPLOYED, dtype: float64

In [96]:
(application_test_merged['DAYS_EMPLOYED'] / -365).describe()

count    47800.000000
mean      -185.142155
std        395.691915
min      -1000.665753
25%          0.832877
50%          3.583562
75%          7.986301
max         47.843836
Name: DAYS_EMPLOYED, dtype: float64

### Something doesn't look right in the time employed variable with a negative 1000 years as the minimum variable. the max seems resonable.

In [97]:
print(np.sum(application_train_merged['DAYS_EMPLOYED'] > 0))
print(np.sum(application_train_merged['DAYS_EMPLOYED'] > 0)/len(application_train_merged['DAYS_EMPLOYED'])*100, 'percent of the train data set has negative length of employment')

52589
18.0682821577904 percent of the train data set has negative length of employment


In [98]:
print(np.sum(application_test_merged['DAYS_EMPLOYED'] > 0))
print(np.sum(application_test_merged['DAYS_EMPLOYED'] > 0)/len(application_test_merged['DAYS_EMPLOYED'])*100, 'percent of the train data set has negative length of employment')

9107
19.052301255230127 percent of the train data set has negative length of employment


In [99]:
# Filling negative employment as zero (these are marked as positive days)

mask = application_train_merged.DAYS_EMPLOYED > 0
column_name = 'DAYS_EMPLOYED'
application_train_merged.loc[mask, column_name] = 0

#test
mask = application_test_merged.DAYS_EMPLOYED > 0
column_name = 'DAYS_EMPLOYED'
application_test_merged.loc[mask, column_name] = 0

In [100]:
(application_train_merged['DAYS_EMPLOYED'] / -365).describe()


count    291057.000000
mean          5.378856
std           6.314441
min          -0.000000
25%           0.816438
50%           3.369863
75%           7.594521
max          49.073973
Name: DAYS_EMPLOYED, dtype: float64

In [101]:
(application_test_merged['DAYS_EMPLOYED'] / -365).describe()

count    47800.000000
mean         5.507698
std          6.270608
min         -0.000000
25%          0.832877
50%          3.583562
75%          7.986301
max         47.843836
Name: DAYS_EMPLOYED, dtype: float64

In [102]:
application_train_merged['Loan_Income_Ratio'] = application_train_merged['AMT_CREDIT_x'] / application_train_merged['AMT_INCOME_TOTAL']
application_test_merged['Loan_Income_Ratio'] = application_test_merged['AMT_CREDIT_x'] / application_test_merged['AMT_INCOME_TOTAL']


In [103]:
application_train_merged.shape

(291057, 151)

In [104]:
application_test_merged.shape

(47800, 150)

In [105]:
# Finding categorical variables
cols = application_train_merged.columns

num_cols = application_train_merged._get_numeric_data().columns

categoriacal_cols=list(set(cols) - set(num_cols))

['NAME_TYPE_SUITE',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'NAME_INCOME_TYPE',
 'ORGANIZATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_EDUCATION_TYPE',
 'FLAG_OWN_CAR',
 'NAME_CONTRACT_TYPE_x',
 'FLAG_OWN_REALTY',
 'CODE_GENDER',
 'WEEKDAY_APPR_PROCESS_START_x']

In [109]:
# Transforming categorical variable into Dummies
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

for col in categoriacal_cols:
    enc.fit(application_train_merged[[col]])
    application_train_merged[[col]] = enc.transform(application_train_merged[[col]])
    application_test_merged[[col]] = enc.transform(application_test_merged[[col]])





In [117]:
application_train_merged[application_train_merged < 0] = 0
application_test_merged[application_test_merged < 0] = 0

application_test_merged.lt(0).any().any()

False

In [118]:
application_train_merged.lt(0).any().any()

False

### Saving updated Merged Files for  EDA

In [119]:
application_train_merged.to_csv('new_application_train_merged.csv',index=False)

In [120]:
application_test_merged.to_csv('new_application_test_merged.csv',index=False)